In [42]:
import pandas as pd
import numpy as np
import gc
import sklearn
import xgboost as xgb
import catboost as ctb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, log_loss, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

In [43]:
para_feat = ['Parameter{0}'.format(i) for i in range(5, 11)]
# para_feat = ['Parameter{0}'.format(i) for i in [5,6,7,10]]
para_feat_all = ['Parameter{0}'.format(i) for i in range(1, 11)]
attr_feat = ['Attribute{0}'.format(i) for i in range(1, 11)]

In [44]:
train = pd.read_csv('final_data/ctb1_train.csv')
test = pd.read_csv('final_data/ctb1_test.csv')
# train = pd.read_csv('final_data/lgb1_train.csv')
# test = pd.read_csv('final_data/lgb1_test.csv')
# train = pd.read_csv('final_data/lgb2_train.csv')
# test = pd.read_csv('final_data/lgb2_test.csv')

ctb1_feat = ['ctb1_Attribute{0}'.format(i) for i in range(4, 7)]
xgb1_feat = ['xgb1_Attribute{0}'.format(i) for i in range(4, 7)]
xgb2_feat = ['xgb2_Attribute{0}'.format(i) for i in range(4, 7)]
lgb1_feat = ['lgb1_Attribute{0}'.format(i) for i in range(4, 7)]
lgb2_feat = ['lgb2_Attribute{0}'.format(i) for i in range(4, 7)]

features = ctb1_feat
# features = lgb1_feat
# features = lgb2_feat
x = train.loc[:, features].values
y = test.loc[:, features].values
# print(y)
y = StandardScaler().fit_transform(y)
x = StandardScaler().fit_transform(x)
pca = PCA(n_components=2)
principaly = pca.fit_transform(y)
principalx = pca.fit_transform(x)
Dfx = pd.DataFrame(data = principalx, columns = ['p1', 'p2'])
Dfy = pd.DataFrame(data = principaly, columns = ['p1', 'p2'])
# print(Df)
test['p1'] = Dfy['p1']
test['p2'] = Dfy['p2']
train['p1'] = Dfx['p1']
train['p2'] = Dfx['p2']

para_feat_discrete = ['Parameter{0}'.format(i) for i in [1,2,3,4,9]]
# feature_name = ctb1_feat + para_feat
# feature_name = para_feat_discrete + ['p1', 'p2']
feature_name = ['p1', 'p2'] + para_feat
# feature_name = para_feat
test.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,...,ctb1_Attribute5,ctb1_Attribute6,ctb1_Attribute7,ctb1_Attribute8,ctb1_Attribute9,ctb1_Attribute10,ctb1_p1,ctb1_p2,p1,p2
0,-0.628417,-0.454090,-0.728591,-1.063207,0.156567,0.090050,-1.414726,0.467028,0.302299,-0.708453,...,0.374567,-0.801534,-0.190086,-0.216354,-0.342409,-0.071401,0.006740,-0.720882,0.355249,-0.937024
1,-1.358538,1.513094,-0.052427,1.472793,-1.561715,-1.773336,-0.221251,-0.927835,2.948223,-0.280689,...,0.319241,2.369127,0.875021,0.846449,1.322568,1.315805,1.245037,1.827654,0.791299,3.000260
2,1.331766,1.073637,4.900365,4.131766,-1.561715,-1.773336,-0.221251,-0.927835,2.948223,-0.280689,...,0.319241,2.369127,0.875021,0.846449,1.322568,1.315805,1.245037,1.827654,0.791299,3.000260
3,-0.152554,-1.742806,2.501081,1.580783,-0.368463,-0.502846,-1.414726,1.251639,-1.285255,-1.991743,...,1.479865,-0.471985,0.776582,0.484669,0.415003,0.166027,1.374844,-0.500816,1.923646,-0.094734
4,1.019031,1.203683,-0.001719,1.223124,-0.320733,-0.333447,-1.414726,1.251639,-1.285255,-1.991743,...,1.429021,-0.498443,0.409922,0.372191,0.413029,0.195827,1.282424,-0.527605,1.887107,-0.121012


In [45]:
X_train = train[feature_name].reset_index(drop=True)
y = train['Quality_label'].reset_index(drop=True).astype(int)
X_test = test[feature_name].reset_index(drop=True)

print(X_train.shape, X_test.shape)
oof = np.zeros((X_train.shape[0],4))
oof_1 = np.zeros((X_train.shape[0],4))
oof_2 = np.zeros((X_train.shape[0],4))
oof_3 = np.zeros((X_train.shape[0],4))

prediction = np.zeros((X_test.shape[0],4))
prediction_1 = np.zeros((X_test.shape[0],4))
prediction_2 = np.zeros((X_test.shape[0],4))
prediction_3 = np.zeros((X_test.shape[0],4))

# model 1
print('Model 1\n\n')
ctb_model_1 = ctb.CatBoostClassifier(iterations=300,learning_rate=0.05,verbose=300, 
                                     early_stopping_rounds=1000,depth=10, 
                                     loss_function='MultiClass',task_type='CPU')

skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True) 
X = X_train
for index, (trn_idx, test_idx) in enumerate(skf.split(X, y)):
    train_x, test_x, train_y, test_y = X.loc[trn_idx], X.loc[test_idx], y.loc[trn_idx], y.loc[test_idx]
    eval_set = [(test_x, test_y)]
    
    ctb_model_1.fit(train_x, train_y, eval_set=eval_set, 
                     # early_stopping_rounds=100, 
                    )
    oof_1 = ctb_model_1.predict_proba(X_test) / 10 + oof_1
    prediction_1 = ctb_model_1.predict_proba(X_test) / 10 + prediction_1


# ctb_model_1.fit(X_train, y ,eval_set=(X_train,y))
# oof_1 = ctb_model_1.predict_proba(X_test)
# prediction_1 = ctb_model_1.predict_proba(X_test)

# model 2
print('Model 2\n\n')
ctb_model_2 = ctb.CatBoostClassifier(iterations=500,learning_rate=0.05,verbose=300, 
                                     early_stopping_rounds=1000,depth=10, 
                                     loss_function='MultiClass',task_type='CPU')

skf = StratifiedKFold(n_splits=10, random_state=43, shuffle=True) 
for index, (trn_idx, test_idx) in enumerate(skf.split(X, y)):
    train_x, test_x, train_y, test_y = X.loc[trn_idx], X.loc[test_idx], y.loc[trn_idx], y.loc[test_idx]
    eval_set = [(test_x, test_y)]
    
    ctb_model_2.fit(train_x, train_y, eval_set=eval_set, 
                     # early_stopping_rounds=100, 
                    )
    oof_2 = ctb_model_2.predict_proba(X_test) / 10 + oof_2
    prediction_2 = ctb_model_2.predict_proba(X_test) / 10 + prediction_2


# ctb_model_2.fit(X_train, y ,eval_set=(X_train,y))
# oof_2 = ctb_model_2.predict_proba(X_test)
# prediction_2 = ctb_model_2.predict_proba(X_test)

# model 3
print('Model 3\n\n')
ctb_model_3 = ctb.CatBoostClassifier(iterations=700,learning_rate=0.05,verbose=300, 
                                     early_stopping_rounds=1000,depth=10, 
                                     loss_function='MultiClass',task_type='CPU')

skf = StratifiedKFold(n_splits=10, random_state=44, shuffle=True) 
for index, (trn_idx, test_idx) in enumerate(skf.split(X, y)):
    train_x, test_x, train_y, test_y = X.loc[trn_idx], X.loc[test_idx], y.loc[trn_idx], y.loc[test_idx]
    eval_set = [(test_x, test_y)]
    
    ctb_model_3.fit(train_x, train_y, eval_set=eval_set, 
                     # early_stopping_rounds=100, 
                    )
    oof_3 = ctb_model_3.predict_proba(X_test) / 10 + oof_3
    prediction_3 = ctb_model_3.predict_proba(X_test) / 10 + prediction_3


# ctb_model_3.fit(X_train, y ,eval_set=(X_train,y))
# oof_3 = ctb_model_3.predict_proba(X_test)
# prediction_3 = ctb_model_3.predict_proba(X_test)


# # model 1
# print('Model 1\n\n')
# lgb_model_1 = lgb.LGBMClassifier(boosting_type="gbdt", num_leaves=23, reg_alpha=10, reg_lambda=5, 
#                                   max_depth=5, n_estimators=300, objective='multiclass', 
#                                   subsample=0.7, colsample_bytree=0.7, subsample_freq=1, min_child_samples=5, 
#                                   learning_rate=0.05, random_state=42
#                                  )

# skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True) 
# X = X_train
# for index, (trn_idx, test_idx) in enumerate(skf.split(X, y)):
#     train_x, test_x, train_y, test_y = X.loc[trn_idx], X.loc[test_idx], y.loc[trn_idx], y.loc[test_idx]
#     eval_set = [(test_x, test_y)]
    
#     lgb_model_1.fit(train_x, train_y, eval_set=eval_set, 
#                      # early_stopping_rounds=100,
#                     verbose = 300,
#                     )
#     oof_1 = lgb_model_1.predict_proba(X_test) / 10 + oof_1
#     prediction_1 = lgb_model_1.predict_proba(X_test) / 10 + prediction_1

# # model 2
# print('Model 2\n\n')
# lgb_model_2 = lgb.LGBMClassifier(boosting_type="gbdt", num_leaves=23, reg_alpha=10, reg_lambda=5, 
#                                   max_depth=5, n_estimators=500, objective='multiclass', 
#                                   subsample=0.7, colsample_bytree=0.7, subsample_freq=1, min_child_samples=5, 
#                                   learning_rate=0.05, random_state=42
#                                  )
                                 
# skf = StratifiedKFold(n_splits=10, random_state=43, shuffle=True) 
# for index, (trn_idx, test_idx) in enumerate(skf.split(X, y)):
#     train_x, test_x, train_y, test_y = X.loc[trn_idx], X.loc[test_idx], y.loc[trn_idx], y.loc[test_idx]
#     eval_set = [(test_x, test_y)]
    
#     lgb_model_2.fit(train_x, train_y, eval_set=eval_set, 
#                      # early_stopping_rounds=100, 
#                     verbose = 300,
#                     )
#     oof_2 = lgb_model_2.predict_proba(X_test) / 10 + oof_2
#     prediction_2 = lgb_model_2.predict_proba(X_test) / 10 + prediction_2

# # model 3
# print('Model 3\n\n')
# lgb_model_3 = lgb.LGBMClassifier(boosting_type="gbdt", num_leaves=23, reg_alpha=10, reg_lambda=5, 
#                                   max_depth=5, n_estimators=700, objective='multiclass', 
#                                   subsample=0.7, colsample_bytree=0.7, subsample_freq=1, min_child_samples=5, 
#                                   learning_rate=0.05, random_state=42
#                                  )

# skf = StratifiedKFold(n_splits=10, random_state=44, shuffle=True) 
# for index, (trn_idx, test_idx) in enumerate(skf.split(X, y)):
#     train_x, test_x, train_y, test_y = X.loc[trn_idx], X.loc[test_idx], y.loc[trn_idx], y.loc[test_idx]
#     eval_set = [(test_x, test_y)]
    
#     lgb_model_3.fit(train_x, train_y, eval_set=eval_set, 
#                      # early_stopping_rounds=100, 
#                     verbose = 300,
#                     )
#     oof_3 = lgb_model_3.predict_proba(X_test) / 10 + oof_3
#     prediction_3 = lgb_model_3.predict_proba(X_test) / 10 + prediction_3

oof = (oof_1 + oof_2 + oof_3) / 3
prediction = (prediction_1 + prediction_2 + prediction_3) / 3
gc.collect()

(6000, 8) (6000, 8)
Model 1


0:	learn: 1.3566466	test: 1.3597761	best: 1.3597761 (0)	total: 59.3ms	remaining: 17.7s
299:	learn: 0.8758016	test: 1.0941224	best: 1.0832306 (113)	total: 18.6s	remaining: 0us

bestTest = 1.083230564
bestIteration = 113

Shrink model to first 114 iterations.
0:	learn: 1.3582364	test: 1.3558826	best: 1.3558826 (0)	total: 97.1ms	remaining: 29s
299:	learn: 0.8871901	test: 0.9946252	best: 0.9917913 (215)	total: 20.3s	remaining: 0us

bestTest = 0.991791321
bestIteration = 215

Shrink model to first 216 iterations.
0:	learn: 1.3583834	test: 1.3596199	best: 1.3596199 (0)	total: 76.1ms	remaining: 22.8s
299:	learn: 0.8803037	test: 1.0789291	best: 1.0665610 (117)	total: 20.1s	remaining: 0us

bestTest = 1.066561024
bestIteration = 117

Shrink model to first 118 iterations.
0:	learn: 1.3586640	test: 1.3591212	best: 1.3591212 (0)	total: 59ms	remaining: 17.7s
299:	learn: 0.8834730	test: 1.0559341	best: 1.0458522 (141)	total: 19.5s	remaining: 0us

bestTest = 1.045852171
b

0

In [46]:
print('logloss',log_loss(pd.get_dummies(y).values, oof))
print('ac',accuracy_score(y, np.argmax(oof,axis=1)))
print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y] - oof))/480))

submit = pd.read_csv('data/submit_example.csv')
sub = test[['Group']].astype(int)
prob_cols = [i for i in submit.columns if i not in ['Group']]
for i, f in enumerate(prob_cols):
    sub[f] = prediction[:, i]
for i in prob_cols:
    sub[i] = sub.groupby('Group')[i].transform('mean')
sub = sub.drop_duplicates()
sub.to_csv("final_sub/ctb1_ctb3x_p1p2_para510_kfold10_1500.csv",index=False)

logloss 1.6172265414101858
ac 0.30133333333333334
mae 0.05319973430578287
